In [47]:
import MeCab
import re

import shinra_util as util

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
s = "ヘマタイトを炭素で還元したり、塩化鉄(III) 溶液や硫化鉄(III) を熱分解して得ることができる。"
node = mecab_param.parseToNode(s)

In [48]:
util.get_compound_list(s)

AttributeError: module 'shinra_util' has no attribute 'get_compound_list'